In [11]:
import json
import numpy as np
from itertools import groupby
from sklearn.cluster import DBSCAN
from collections import Counter
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

In [2]:
class News:
    def __init__(self, id, date, title, content, url, siteType):
        self.id = id
        self.date = date
        self.title = title
        self.content = content
        self.url = url
        self.siteType = siteType
    
    @classmethod
    def from_json(cls, json_str):
        json_dict = json.loads(json_str)
        return cls(**json_dict)

In [3]:
news = []
with open('/data/kasandra/year/all.normalized.json', encoding="utf8") as f:
    for line in f:
        news.append(News.from_json(line))

In [4]:
words = []
for n in news:
    words.extend(n.content.split())
counts = Counter(words)
one_time = [k for k, v in dict(counts).items() if v == 1]
print("total words: %s" % (len(words) - len(one_time)))

total words: 47479434


In [5]:
stopwords = set(one_time)

In [6]:
def zip_news(n,l):
    return list(map(assign_label_to_news, zip(n, l)))

def assign_label_to_news(tuplezz):
    (nws, lbl) = tuplezz
    nws.label = lbl.item()
    return nws

def filter_words(text):
    words_list = text.split()
    newWords = [x for x in words_list if len(x) > 3]
    return " ".join(newWords)

def print_clusters(cluster_news, clustre_labels):
    newsLabels = zip_news(cluster_news, clustre_labels)
    newsLabels = sorted(newsLabels, key=lambda n: n.label)
    for label, group in groupby(newsLabels, lambda n: n.label):
        groupList = list(group)
        print("Cluster: %s, count news: %s, titles:" % (label, len(groupList)))
        for gr in groupList:
            print("\t" + gr.title)
            
def print_topics(components, feature_names, n_top_words):
    for topic_idx, topic in enumerate(components):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

# Март 2015

In [7]:
mart_start = 1425157200000
mart_end = 1427835600000

In [8]:
mart_news = list(filter(lambda x: x.date > mart_start and x.date < mart_end, news))
mart_content = [filter_words(x.content) for x in mart_news]
print("count mart news: %s" % len(mart_content))

count mart news: 2684


In [46]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True, tokenizer=lambda text: text.split(" "), stop_words=stopwords, max_df=3, min_df=1) # , ngram_range=(1, 3)

tfidf_matrix = tfidf_vectorizer.fit_transform(mart_content)
print("vocabulary size: %s" % len(tfidf_vectorizer.vocabulary_))

vocabulary size: 23656


# DBScan

In [48]:
X = (tfidf_matrix * tfidf_matrix.T).A #cosine
db = DBSCAN(eps=1.1, min_samples=10).fit(X)
labels = db.labels_

print('count clusters: %d' % (len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)))

labels = db.labels_
print("-1: %s, 0: %s" % (labels.tolist().count(-1), labels.tolist().count(0)))

count clusters: 1
-1: 478, 0: 2206


In [ ]:
print_clusters(mart_news, labels)

## Сохранение данных

In [48]:
news_clusters = {}
for i in range(0, len(labels)):
    n = mart_news[i]
    l = labels[i]
    news_clusters[n.id] = int(l)

In [49]:
cluster_file = '/data/kasandra/year/result/50_kmens.json'

with open(cluster_file, encoding="utf8", mode="w") as f:
    d_json = json.dumps(news_clusters, ensure_ascii=False)
    f.write(d_json + '\n')